In [17]:
!pip install asyncio_pool

In [5]:
!pip install aiofiles

In [3]:
!pip install asyncio_pool

In [1]:
import requests
import os
from pathlib import Path
import tqdm


import asyncio
import aiohttp
import aiofiles
from asyncio_pool import AioPool
from tqdm.asyncio import tqdm_asyncio

In [9]:
class SPBU_scrapper():
    WORKS_LIMIT = 2000
        
    def check_downloaded(self):
        self.works_to_download = [f"work_{i}.pdf" for i in range(1, self.WORKS_LIMIT)]        
        
        for path in os.listdir(f"{self.data_dir}/spbu/pdf"):
            if os.path.isfile(os.path.join(f"{self.data_dir}/spbu/pdf", path)):
                self.works_to_download.remove(path)

    def __init__(self, data_dir="data"):
        self.data_dir = data_dir
        path = Path(f"{data_dir}/spbu/pdf").mkdir(parents=True, exist_ok=True)
        
        self.check_downloaded()
        
        print(f"Found {self.WORKS_LIMIT - len(self.works_to_download)} pdfs")
        print()
    
    async def parse(self, is_async=True, pool_size=20):

        error_works = []

        async def fetch_file(work):
            work_id = work[5:-4]
            url = f"https://se.math.spbu.ru/thesis_download?thesis_id={work_id}"
            
            async with aiohttp.ClientSession() as session:
                async with session.get(url) as resp:
                    if resp.status == 200:
                        data = await resp.read()
                        async with aiofiles.open(
                            os.path.join(self.data_dir, 'spbu/pdf', work), "wb"
                        ) as outfile:
                            await outfile.write(data)
                    else:
                        print(work_id, resp)
        
        if is_async:
            pool = AioPool(size=pool_size)
            await pool.map(fetch_file, self.works_to_download)
            
            to_download = len(self.works_to_download)
            self.check_downloaded()
            print(f"Downloaded {to_download - len(self.works_to_download)}/{to_download}")
            
        else:
            for work in tqdm.tqdm(self.works_to_download):
                await fetch_file(work)
            
        self.check_downloaded()

In [11]:
scrapper = SPBU_scrapper()
await scrapper.parse(is_async=True, pool_size=5)

Found 799 pdfs

29 <ClientResponse(https://se.math.spbu.ru/theses_search) [404 NOT FOUND]>
<CIMultiDictProxy('Server': 'nginx/1.10.3 (Ubuntu)', 'Date': 'Fri, 21 Jul 2023 19:07:46 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Strict-Transport-Security': 'max-age=63072000', 'Content-Encoding': 'gzip')>

189 <ClientResponse(https://se.math.spbu.ru/theses_search) [404 NOT FOUND]>
<CIMultiDictProxy('Server': 'nginx/1.10.3 (Ubuntu)', 'Date': 'Fri, 21 Jul 2023 19:07:46 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Strict-Transport-Security': 'max-age=63072000', 'Content-Encoding': 'gzip')>

39 <ClientResponse(https://se.math.spbu.ru/theses_search) [404 NOT FOUND]>
<CIMultiDictProxy('Server': 'nginx/1.10.3 (Ubuntu)', 'Date': 'Fri, 21 Jul 2023 19:07:46 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Stric

382 <ClientResponse(https://se.math.spbu.ru/thesis_download?thesis_id=382) [500 INTERNAL SERVER ERROR]>
<CIMultiDictProxy('Server': 'nginx/1.10.3 (Ubuntu)', 'Date': 'Fri, 21 Jul 2023 19:07:48 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Content-Length': '290', 'Connection': 'keep-alive', 'Strict-Transport-Security': 'max-age=63072000')>

386 <ClientResponse(https://se.math.spbu.ru/thesis_download?thesis_id=386) [500 INTERNAL SERVER ERROR]>
<CIMultiDictProxy('Server': 'nginx/1.10.3 (Ubuntu)', 'Date': 'Fri, 21 Jul 2023 19:07:48 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Content-Length': '290', 'Connection': 'keep-alive', 'Strict-Transport-Security': 'max-age=63072000')>

384 <ClientResponse(https://se.math.spbu.ru/thesis_download?thesis_id=384) [500 INTERNAL SERVER ERROR]>
<CIMultiDictProxy('Server': 'nginx/1.10.3 (Ubuntu)', 'Date': 'Fri, 21 Jul 2023 19:07:48 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Content-Length': '290', 'Connection': 'keep-alive', 'Strict-Trans

439 <ClientResponse(https://se.math.spbu.ru/thesis_download?thesis_id=439) [500 INTERNAL SERVER ERROR]>
<CIMultiDictProxy('Server': 'nginx/1.10.3 (Ubuntu)', 'Date': 'Fri, 21 Jul 2023 19:07:54 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Content-Length': '290', 'Connection': 'keep-alive', 'Strict-Transport-Security': 'max-age=63072000')>

447 <ClientResponse(https://se.math.spbu.ru/thesis_download?thesis_id=447) [500 INTERNAL SERVER ERROR]>
<CIMultiDictProxy('Server': 'nginx/1.10.3 (Ubuntu)', 'Date': 'Fri, 21 Jul 2023 19:07:54 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Content-Length': '290', 'Connection': 'keep-alive', 'Strict-Transport-Security': 'max-age=63072000')>

452 <ClientResponse(https://se.math.spbu.ru/thesis_download?thesis_id=452) [500 INTERNAL SERVER ERROR]>
<CIMultiDictProxy('Server': 'nginx/1.10.3 (Ubuntu)', 'Date': 'Fri, 21 Jul 2023 19:07:54 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Content-Length': '290', 'Connection': 'keep-alive', 'Strict-Trans

524 <ClientResponse(https://se.math.spbu.ru/theses_search) [404 NOT FOUND]>
<CIMultiDictProxy('Server': 'nginx/1.10.3 (Ubuntu)', 'Date': 'Fri, 21 Jul 2023 19:07:58 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Strict-Transport-Security': 'max-age=63072000', 'Content-Encoding': 'gzip')>

526 <ClientResponse(https://se.math.spbu.ru/theses_search) [404 NOT FOUND]>
<CIMultiDictProxy('Server': 'nginx/1.10.3 (Ubuntu)', 'Date': 'Fri, 21 Jul 2023 19:07:58 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Strict-Transport-Security': 'max-age=63072000', 'Content-Encoding': 'gzip')>

527 <ClientResponse(https://se.math.spbu.ru/theses_search) [404 NOT FOUND]>
<CIMultiDictProxy('Server': 'nginx/1.10.3 (Ubuntu)', 'Date': 'Fri, 21 Jul 2023 19:07:58 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Strict-Transport-Se

559 <ClientResponse(https://se.math.spbu.ru/thesis_download?thesis_id=559) [500 INTERNAL SERVER ERROR]>
<CIMultiDictProxy('Server': 'nginx/1.10.3 (Ubuntu)', 'Date': 'Fri, 21 Jul 2023 19:07:58 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Content-Length': '290', 'Connection': 'keep-alive', 'Strict-Transport-Security': 'max-age=63072000')>

561 <ClientResponse(https://se.math.spbu.ru/theses_search) [404 NOT FOUND]>
<CIMultiDictProxy('Server': 'nginx/1.10.3 (Ubuntu)', 'Date': 'Fri, 21 Jul 2023 19:07:59 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Strict-Transport-Security': 'max-age=63072000', 'Content-Encoding': 'gzip')>

569 <ClientResponse(https://se.math.spbu.ru/thesis_download?thesis_id=569) [500 INTERNAL SERVER ERROR]>
<CIMultiDictProxy('Server': 'nginx/1.10.3 (Ubuntu)', 'Date': 'Fri, 21 Jul 2023 19:08:00 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Content-Length': '290', 'Connection': 'keep-alive', 'Stric

...

1826 <ClientResponse(https://se.math.spbu.ru/theses_search) [404 NOT FOUND]>
<CIMultiDictProxy('Server': 'nginx/1.10.3 (Ubuntu)', 'Date': 'Fri, 21 Jul 2023 19:08:58 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Strict-Transport-Security': 'max-age=63072000', 'Content-Encoding': 'gzip')>

1827 <ClientResponse(https://se.math.spbu.ru/theses_search) [404 NOT FOUND]>
<CIMultiDictProxy('Server': 'nginx/1.10.3 (Ubuntu)', 'Date': 'Fri, 21 Jul 2023 19:08:58 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Strict-Transport-Security': 'max-age=63072000', 'Content-Encoding': 'gzip')>

1828 <ClientResponse(https://se.math.spbu.ru/theses_search) [404 NOT FOUND]>
<CIMultiDictProxy('Server': 'nginx/1.10.3 (Ubuntu)', 'Date': 'Fri, 21 Jul 2023 19:08:58 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Strict-Transport

1852 <ClientResponse(https://se.math.spbu.ru/theses_search) [404 NOT FOUND]>
<CIMultiDictProxy('Server': 'nginx/1.10.3 (Ubuntu)', 'Date': 'Fri, 21 Jul 2023 19:08:58 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Strict-Transport-Security': 'max-age=63072000', 'Content-Encoding': 'gzip')>

1855 <ClientResponse(https://se.math.spbu.ru/theses_search) [404 NOT FOUND]>
<CIMultiDictProxy('Server': 'nginx/1.10.3 (Ubuntu)', 'Date': 'Fri, 21 Jul 2023 19:08:58 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Strict-Transport-Security': 'max-age=63072000', 'Content-Encoding': 'gzip')>

1854 <ClientResponse(https://se.math.spbu.ru/theses_search) [404 NOT FOUND]>
<CIMultiDictProxy('Server': 'nginx/1.10.3 (Ubuntu)', 'Date': 'Fri, 21 Jul 2023 19:08:58 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Strict-Transport

1895 <ClientResponse(https://se.math.spbu.ru/theses_search) [404 NOT FOUND]>
<CIMultiDictProxy('Server': 'nginx/1.10.3 (Ubuntu)', 'Date': 'Fri, 21 Jul 2023 19:08:59 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Strict-Transport-Security': 'max-age=63072000', 'Content-Encoding': 'gzip')>

1896 <ClientResponse(https://se.math.spbu.ru/theses_search) [404 NOT FOUND]>
<CIMultiDictProxy('Server': 'nginx/1.10.3 (Ubuntu)', 'Date': 'Fri, 21 Jul 2023 19:08:59 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Strict-Transport-Security': 'max-age=63072000', 'Content-Encoding': 'gzip')>

1898 <ClientResponse(https://se.math.spbu.ru/theses_search) [404 NOT FOUND]>
<CIMultiDictProxy('Server': 'nginx/1.10.3 (Ubuntu)', 'Date': 'Fri, 21 Jul 2023 19:08:59 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Strict-Transport

1940 <ClientResponse(https://se.math.spbu.ru/theses_search) [404 NOT FOUND]>
<CIMultiDictProxy('Server': 'nginx/1.10.3 (Ubuntu)', 'Date': 'Fri, 21 Jul 2023 19:08:59 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Strict-Transport-Security': 'max-age=63072000', 'Content-Encoding': 'gzip')>

1942 <ClientResponse(https://se.math.spbu.ru/theses_search) [404 NOT FOUND]>
<CIMultiDictProxy('Server': 'nginx/1.10.3 (Ubuntu)', 'Date': 'Fri, 21 Jul 2023 19:08:59 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Strict-Transport-Security': 'max-age=63072000', 'Content-Encoding': 'gzip')>

1941 <ClientResponse(https://se.math.spbu.ru/theses_search) [404 NOT FOUND]>
<CIMultiDictProxy('Server': 'nginx/1.10.3 (Ubuntu)', 'Date': 'Fri, 21 Jul 2023 19:08:59 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Strict-Transport

1989 <ClientResponse(https://se.math.spbu.ru/theses_search) [404 NOT FOUND]>
<CIMultiDictProxy('Server': 'nginx/1.10.3 (Ubuntu)', 'Date': 'Fri, 21 Jul 2023 19:09:00 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Strict-Transport-Security': 'max-age=63072000', 'Content-Encoding': 'gzip')>

1992 <ClientResponse(https://se.math.spbu.ru/theses_search) [404 NOT FOUND]>
<CIMultiDictProxy('Server': 'nginx/1.10.3 (Ubuntu)', 'Date': 'Fri, 21 Jul 2023 19:09:00 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Strict-Transport-Security': 'max-age=63072000', 'Content-Encoding': 'gzip')>

1991 <ClientResponse(https://se.math.spbu.ru/theses_search) [404 NOT FOUND]>
<CIMultiDictProxy('Server': 'nginx/1.10.3 (Ubuntu)', 'Date': 'Fri, 21 Jul 2023 19:09:00 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Strict-Transport